In [1]:
import mygrad as mg
#loss = mg.mean(mg.maximum(0, margin - y * (x1 - x2)))
import numpy as np
from gensim.models import KeyedVectors
from cogworks_data.language import get_data_path
from pathlib import Path
import json
import re, string
from collections import Counter
import numpy as np
import math
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import torch 

In [2]:
# load COCO metadata
filename = get_data_path("captions_train2014.json")
with Path(filename).open() as f:
    coco_data = json.load(f)

In [3]:
filename = "glove.6B.200d.txt.w2v"

# this takes a while to load -- keep this in mind when designing your capstone project
glove = KeyedVectors.load_word2vec_format(get_data_path(filename), binary=False)

In [4]:
punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
def strip_punc(corpus):
    return punc_regex.sub('', corpus)

text = "Apples rule. Apples are the best. Truly, they are. Truly... Truly"
text = strip_punc(text).lower()
tokenized = text.split()
tokens = Counter(tokenized)
#descriptor = np.array([tokens[word] for word in sorted(tokens)], dtype=float)
#descriptor
#tokens

**Embed Caption**

In [5]:
def to_counter(doc):
    return Counter(strip_punc(doc).lower().split())

In [6]:
def to_vocab(counters, k=None):
    output = set()
    for i in counters:
        output.add(i)
    #output = set(output)
    return sorted(output)

In [7]:
def to_idf(vocab, counters):
    N = len(counters)
    nt = [sum(1 if term in counter else 0 for counter in counters) for term in vocab]
    nt = np.array(nt, dtype=float)

    return np.log10(N/nt)    

In [8]:
def embed_caption(caption):
    caption = to_counter(caption)
    caption_idf = to_idf(to_vocab(caption), caption)
    #caption_glove = [glove[i] for i in caption]
    caption_glove=[]
    for i in range(len(caption)):
        try:
            caption_glove.append(glove[caption[i]])
        except:
            caption_idf.pop(i)
            continue
    
    dots=[]
    for i in range(len(caption_idf)):
        dots.append(np.dot(caption_idf[i], caption_glove[i]))        
    summation = sum(dots)
    unit = summation / math.sqrt(sum(summation**2))

    return unit

**Image Descriptor**

In [9]:
with Path(get_data_path('resnet18_features.pkl')).open('rb') as f:
    resnet18_features = pickle.load(f)

In [10]:
resnet18_features[57870].shape

(1, 512)

In [11]:
coco_data["annotations"][0]

{'image_id': 318556,
 'id': 48,
 'caption': 'A very clean and well decorated empty bathroom'}

**Creating Training Data**

In [12]:
len(resnet18_features)

82612

In [13]:
len(coco_data["images"])

82783

In [33]:
training_data = [] 
caption_vectors = []
good_image_vectors = []
bad_image_vectors = []
index=0

for i in range(len(list(resnet18_features.keys())[:500])):
    caption_vectors.append(np.array(embed_caption(coco_data["annotations"][i]["caption"])))
    good_image_vectors.append(np.array(resnet18_features[list(resnet18_features.keys())[i]]))
    """
    while True:
        random_key = list(resnet18_features.keys())[np.random.randint(len(resnet18_features.keys()))]
        if (random_key!=coco_data["annotations"][j]["image_id"]):
            break
    bad_image_vectors.append(resnet18_features[random_key])
    """
    random_key = list(resnet18_features.keys())[np.random.randint(len(resnet18_features.keys()))]
    if (random_key!=coco_data["annotations"][i]["image_id"]):
        bad_image_vectors.append(np.array(resnet18_features[random_key]))
        training_data.append((coco_data["annotations"][i]["id"], coco_data["annotations"][i]["image_id"], random_key)) #(true-caption-ID, true-image-ID, confusor-image-ID)


In [38]:
print(np.array(caption_vectors).shape)
print(np.array(good_image_vectors).shape)
print(np.array(bad_image_vectors).shape)


(500, 200)
(500, 1, 512)
(500, 1, 512)


In [15]:
training_data

[(48, 318556, 298473),
 (67, 116100, 175952),
 (126, 318556, 581582),
 (148, 116100, 478266),
 (173, 379340, 477046),
 (188, 379340, 468830),
 (219, 318556, 115251),
 (255, 318556, 288651),
 (272, 134754, 391600),
 (288, 538480, 297018),
 (314, 476220, 297868),
 (328, 299675, 188556),
 (352, 32275, 457324),
 (411, 302443, 209494),
 (425, 134754, 473607),
 (519, 25470, 114204),
 (572, 513461, 412612),
 (653, 18691, 273879),
 (668, 285579, 128942),
 (674, 266366, 513944),
 (710, 226658, 451341),
 (715, 299675, 532525),
 (729, 124567, 27395),
 (750, 293605, 305527),
 (760, 321194, 280589),
 (764, 479495, 267531),
 (818, 539984, 336959),
 (819, 28231, 74993),
 (873, 538480, 65831),
 (877, 350235, 82896),
 (928, 314265, 514204),
 (946, 389006, 71552),
 (1022, 134754, 424088),
 (1029, 31813, 558044),
 (1098, 538480, 89318),
 (1124, 465049, 374288),
 (1161, 360334, 23098),
 (1184, 524679, 366058),
 (1193, 279672, 50706),
 (1199, 385716, 269228),
 (1231, 44816, 261051),
 (1273, 116100, 52619),

**Model**

In [16]:
#caption vector
#embed_caption(coco_data["annotations"]["48"])]
coco_data["annotations"][0]

{'image_id': 318556,
 'id': 48,
 'caption': 'A very clean and well decorated empty bathroom'}

In [17]:
#descriptor vectors
resnet18_features[57870]

array([[1.04235423e+00, 6.82473838e-01, 9.16481733e-01, 2.32561696e-02,
        1.45509019e-01, 7.50707984e-01, 1.54418981e+00, 8.72745886e-02,
        2.67963076e+00, 3.70128679e+00, 2.69317198e+00, 5.91564715e-01,
        2.74772495e-02, 4.67028096e-02, 4.01344031e-01, 1.49007127e-01,
        1.86075401e+00, 1.31343961e+00, 4.01950441e-02, 1.01394176e+00,
        1.59402609e+00, 3.84880638e+00, 1.45292222e+00, 8.55284631e-01,
        1.12077498e+00, 2.91885662e+00, 6.87212348e-02, 1.92611444e+00,
        9.75241363e-01, 7.45405018e-01, 5.55366576e-01, 7.82756627e-01,
        2.97761202e-01, 1.86988008e+00, 1.57985020e+00, 7.58939236e-02,
        4.28391933e-01, 1.61274552e+00, 1.61727130e+00, 9.25690651e-01,
        5.45925163e-02, 1.61366329e-01, 2.14940667e+00, 7.87875533e-01,
        3.76012659e+00, 1.25876856e+00, 2.67516875e+00, 2.98997712e+00,
        5.63193202e-01, 5.04034340e-01, 2.94659100e-03, 1.22843899e-01,
        1.60765755e+00, 8.74048173e-01, 1.57004848e-01, 1.660890

In [70]:
def loss_func(caption_vect, img_good, img_bad, Wembed):

    #print(len(img_good))
    #print(Wembed.shape)
        
    img_good_embed = np.matmul(img_good, Wembed.detach().numpy())
    img_bad_embed = np.matmul(img_bad, Wembed.detach().numpy())

    img_good_embed = np.reshape(img_good_embed, (-1,200))
    img_bad_embed = np.reshape(img_bad_embed, (-1,200))

    #print(img_good_embed.shape)
    #print(img_bad_embed.shape)

    sim_to_good = np.matmul(caption_vect, np.transpose(img_good_embed)) 
    sim_to_bad = np.matmul(caption_vect, np.transpose(img_bad_embed))
    #sim_to_good = mg.einsum("ab, acb -> b", caption_vect, img_good_embed)
    #sim_to_bad = mg.einsum("ab, acb -> b",caption_vect, img_bad_embed)

    #print(sim_to_good)
    #print(sim_to_bad)
    
    #sim_to_good = cosine_similarity(text, img_good)
    #sim_to_bad = cosine_similarity(text, img_bad)
    
    y = np.ones(len(sim_to_good)) #1 or -1, 1 makes it so that its x1 - x2 (vice versa with -1).
    
    return mg.nnet.losses.margin_ranking_loss(sim_to_good, sim_to_bad, y, margin = 0.25)

In [71]:
Wembed = torch.nn.Parameter(torch.randn(512, 200), requires_grad=True)
optimizer = torch.optim.SGD([Wembed], lr=0.1, momentum=0.9)

In [74]:
def train(vectors_stuff, good_img_stuff, bad_img_stuff, Wembed, optimizer, num_epochs, batch_size=32):
    for epoch in range(num_epochs):
        for i in range(0, len(vectors_stuff), batch_size):
            batch_vect = vectors_stuff[i:i+batch_size]
            batch_good_img = good_img_stuff[i:i+batch_size]
            batch_bad_img = bad_img_stuff[i:i+batch_size]
    
            loss = loss_func(batch_vect, batch_good_img, batch_bad_img, Wembed)  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(Wembed)
    return(Wembed)

In [ ]:
#vectors_stuff = caption_vectors[:len(caption_vectors)//4 * 5]
#good_img_stuff = good_image_vectors[:len(caption_vectors)//4 * 5]
#bad_img_stuff = bad_image_vectors[:len(caption_vectors)//4 * 5]

train(caption_vectors, good_image_vectors, bad_image_vectors, Wembed, optimizer, num_epochs=100)


32
torch.Size([512, 200])
(32, 200)
(32, 200)
Parameter containing:
tensor([[-0.0372,  0.0861, -1.3020,  ..., -0.2101,  1.7872,  1.0143],
        [ 0.9616,  0.9060,  0.9483,  ...,  0.0625, -1.1388, -0.3605],
        [-1.4859, -0.4322, -2.2124,  ...,  2.1044, -0.0913, -1.0227],
        ...,
        [-0.0471, -0.6792,  0.4704,  ...,  0.2533,  0.4664,  0.4605],
        [ 1.2307, -0.1989, -2.0615,  ..., -0.8806, -0.0749, -0.4351],
        [ 0.4646,  0.8581,  0.8500,  ..., -0.9729,  0.0828,  2.0122]],
       requires_grad=True)
32
torch.Size([512, 200])
(32, 200)
(32, 200)
Parameter containing:
tensor([[-0.0372,  0.0861, -1.3020,  ..., -0.2101,  1.7872,  1.0143],
        [ 0.9616,  0.9060,  0.9483,  ...,  0.0625, -1.1388, -0.3605],
        [-1.4859, -0.4322, -2.2124,  ...,  2.1044, -0.0913, -1.0227],
        ...,
        [-0.0471, -0.6792,  0.4704,  ...,  0.2533,  0.4664,  0.4605],
        [ 1.2307, -0.1989, -2.0615,  ..., -0.8806, -0.0749, -0.4351],
        [ 0.4646,  0.8581,  0.8500,  ...